#### 인공지능 기초 설계 
##### pytorch 

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from pytorch_class import NeuralNetwork
from pytorch_train import train
from pytorch_test import test

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

model = NeuralNetwork().to(device) ##cpu에서 게산이 되는데 사용하면 
print(model)                       ##예를 들어 GPU메모리로 보내줄 수 있음 

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

Using cpu device


AttributeError: 'NeuralNetwork' object has no attribute 'parameters'

In [34]:
##############################################################################
# The training process is conducted over several iterations (*epochs*). During each epoch, the model learns
# parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
# accuracy increase and the loss decrease with every epoch.

epochs = 5

train_acc_list = []
test_acc_list = []

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_acc = train(train_dataloader, model, loss_fn, optimizer)
    test_acc = test(test_dataloader, model, loss_fn)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
print("Done!")

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Epoch 1
-------------------------------
loss: 2.302542  [   64/60000]
loss: 2.155196  [ 6464/60000]
loss: 1.784626  [12864/60000]
loss: 1.478585  [19264/60000]
loss: 1.123590  [25664/60000]
loss: 1.023636  [32064/60000]
loss: 0.986988  [38464/60000]
loss: 0.865731  [44864/60000]
loss: 0.866138  [51264/60000]
loss: 0.801439  [57664/60000]
Test Error: 
 Accuracy: 71.3%, Avg loss: 0.789585 

Epoch 2
-------------------------------
loss: 0.789515  [   64/60000]
loss: 0.841486  [ 6464/60000]
loss: 0.572359  [12864/60000]
loss: 0.780757  [19264/60000]
loss: 0.647916  [25664/60000]
loss: 0.625259  [32064/60000]
loss: 0.702828  [38464/60000]
loss: 0.688372  [44864/60000]
loss: 0.701756  [51264/60000]
loss: 0.622198  [57664/60000]
Test Error: 
 Accuracy: 78.2%, Avg loss: 0.630164 

Epoch 3
-------------------------------
loss: 0.558174  [   64/60000]
loss: 0.645770  [ 6464/60000]
loss: 0.429080  [12864/60000]
loss: 0.666724  [19264/60000]
loss: 0.570290  [25664/60000]
loss: 0.551412  [32064/600

In [ ]:
# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()